<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/TrainTest_2_3class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

#### สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [5]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150l_dir = os.path.join(data_dir,'CA150')
All_3class_dir = os.path.join(data_dir,'All_3class')

In [6]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [7]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [8]:
lungcancer = []
for i in range(len(os.listdir(CA150l_dir))):
    img = os.listdir(CA150l_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [9]:
data_train2 = pd.concat([tnormal, ttuberculosis])
data_train2.head(3)

,ImageName,NameType,NumberType
0,124.jpg,Normal,0
1,128.jpg,Normal,0
2,151.jpg,Normal,0


In [10]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca26.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


# 2 class
*   #1 output 1 มิติ  ความน่าจะเป็น นำมาแปลง 0,1
*   #2 output 2 มิติ  ความน่าจะเป็นที่จะเป็นคลาส 0 หรือ 1 



In [11]:
train_img_NorTB = []
for ImageName in tqdm(data_train2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = All_3class_dir +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img_NorTB.append(img)

100%|██████████| 301/301 [02:49<00:00,  1.77it/s]


## 2) output 2 มิติ : ค่า predict คือ ความน่าจะเป็น 2 ค่า 
(ความน่าจะเป็นที่จะเป็นคลาส 0 และ 1)

In [12]:
data2_2 = np.array(train_img_NorTB, dtype="float32") / 255.0 

In [13]:
labels2_2 = np.array(data_train2[['NameType']] )            # Y  มี  d=2  (301, 1)
labels2_2.shape = (len(labels2_2),)                                # Y  มี  d=1  (301, )

le2_2 = LabelEncoder()                                                     # จะแปลงชื่อ class เป็นตัวเลข
labels2_2 = le2_2.fit_transform(labels2_2)                          # Y  มี  d=1  (301,)

labels2_2 = to_categorical(labels2_2, num_classes=2)    # แปลงเป็นเลข ดัมมี่ (1เป็น 1,0 ----- 0 เป็น  0,1)
                                                                                 # Y  มี  d=2  (301, 2)    
(trainX2_2, testX2_2, trainY2_2, testY2_2 ) = train_test_split(data2_2, labels2_2,
	                                                            test_size=0.25, random_state=40)

In [14]:
le2_2.classes_                  # มีค่า ['Normal', 'Tuberculosis']

array(['Normal', 'Tuberculosis'], dtype=object)

### train2_2

In [15]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [16]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [17]:
base_model2_2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

74850304/74836368 [==============================] - 1s 0us/step


In [18]:
base_model2_2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2_2.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # ลองแก้ unit จาก 1 เป็น 2
model2_2 = tf.keras.models.Model(inputs=base_model2_2.input, outputs=prediction_layer)

In [19]:
model2_2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    mode="max"   )

In [21]:
H2_2 = model2_2.fit(
	x=aug.flow(trainX2_2, trainY2_2, batch_size=BS),
	validation_data=(testX2_2, testY2_2),
	steps_per_epoch=len(trainX2_2) // BS,
	epochs=30, callbacks=[callback])

Epoch 1/30
28/28 [==============================] - 78s 2s/step - loss: 0.7507 - accuracy: 0.4470 - val_loss: 0.7492 - val_accuracy: 0.4211
Epoch 2/30
28/28 [==============================] - 67s 2s/step - loss: 0.7138 - accuracy: 0.5438 - val_loss: 0.7343 - val_accuracy: 0.4605
Epoch 3/30
28/28 [==============================] - 67s 2s/step - loss: 0.6849 - accuracy: 0.5853 - val_loss: 0.7212 - val_accuracy: 0.5132
Epoch 4/30
28/28 [==============================] - 68s 2s/step - loss: 0.6933 - accuracy: 0.5991 - val_loss: 0.7026 - val_accuracy: 0.5526
Epoch 5/30
28/28 [==============================] - 70s 3s/step - loss: 0.6877 - accuracy: 0.5714 - val_loss: 0.6874 - val_accuracy: 0.5658
Epoch 6/30
28/28 [==============================] - 71s 3s/step - loss: 0.6594 - accuracy: 0.6866 - val_loss: 0.6804 - val_accuracy: 0.5789
Epoch 7/30
28/28 [==============================] - 69s 2s/step - loss: 0.6485 - accuracy: 0.6728 - val_loss: 0.6605 - val_accuracy: 0.6447
Epoch 8/30
28/28 [==

### test

In [22]:
predictions2_2 = model2_2.predict(testX2_2, batch_size=BS)

In [23]:
predictions2_2.ndim

2

In [24]:
predictions2_2[:5]

array([[0.747986  , 0.4587629 ],
       [0.71529895, 0.37289017],
       [0.2162846 , 0.7339208 ],
       [0.42456186, 0.3854583 ],
       [0.5317864 , 0.5034192 ]], dtype=float32)

In [25]:
print(classification_report(testY2_2.argmax(axis=1),	
                            predictions2_2.argmax(axis=1),  
							target_names=le2_2.classes_ ))

              precision    recall  f1-score   support

      Normal       0.78      0.95      0.85        40
Tuberculosis       0.93      0.69      0.79        36

    accuracy                           0.83        76
   macro avg       0.85      0.82      0.82        76
weighted avg       0.85      0.83      0.83        76



In [26]:
confusion_matrix(testY2_2.argmax(axis=1),	
                        predictions2_2.argmax(axis=1) )

array([[38,  2],
       [11, 25]])

## Fine tuning

In [27]:
len(base_model2_2.layers)

707

In [28]:
base_model2_2.trainable = True
for layer in base_model2_2.layers[:300]:    #fix w & bias in layer 0-300
    layer.trainable = False

In [29]:
model2_2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [30]:
H2_2Finetune = model2_2.fit(
	x=aug.flow(trainX2_2, trainY2_2, batch_size=BS),
	validation_data=(testX2_2, testY2_2),
	steps_per_epoch=len(trainX2_2) // BS,
	epochs=30, callbacks=[callback])

Epoch 1/30
28/28 [==============================] - 218s 7s/step - loss: 0.4655 - accuracy: 0.7834 - val_loss: 0.3731 - val_accuracy: 0.8289
Epoch 2/30
28/28 [==============================] - 173s 6s/step - loss: 0.2300 - accuracy: 0.9124 - val_loss: 0.4488 - val_accuracy: 0.7895
Epoch 3/30
28/28 [==============================] - 135s 5s/step - loss: 0.2837 - accuracy: 0.8848 - val_loss: 0.1586 - val_accuracy: 0.9737
Epoch 4/30
28/28 [==============================] - 134s 5s/step - loss: 0.2891 - accuracy: 0.8710 - val_loss: 0.2973 - val_accuracy: 0.8816
Epoch 5/30
28/28 [==============================] - 134s 5s/step - loss: 0.2273 - accuracy: 0.9124 - val_loss: 0.3212 - val_accuracy: 0.8947
Epoch 6/30
28/28 [==============================] - 134s 5s/step - loss: 0.1430 - accuracy: 0.9401 - val_loss: 0.2052 - val_accuracy: 0.9474
Epoch 7/30
28/28 [==============================] - 134s 5s/step - loss: 0.1726 - accuracy: 0.9585 - val_loss: 0.1322 - val_accuracy: 0.9605
Epoch 8/30
28

###  Fine tuning test

In [31]:
predictions2_2 = model2_2.predict(testX2_2, batch_size=BS)

In [32]:
predictions2_2.ndim

2

In [33]:
predictions2_2[:5]

array([[9.9902034e-01, 8.6322427e-04],
       [9.9992716e-01, 1.1755234e-04],
       [9.3636692e-02, 9.1446686e-01],
       [8.0407238e-01, 1.0624474e-01],
       [9.1175961e-01, 1.1564425e-01]], dtype=float32)

In [34]:
print(classification_report(testY2_2.argmax(axis=1),	
                            predictions2_2.argmax(axis=1),  
							target_names=le2_2.classes_ ))

              precision    recall  f1-score   support

      Normal       0.73      1.00      0.84        40
Tuberculosis       1.00      0.58      0.74        36

    accuracy                           0.80        76
   macro avg       0.86      0.79      0.79        76
weighted avg       0.86      0.80      0.79        76



In [35]:
confusion_matrix(testY2_2.argmax(axis=1),	
                        predictions2_2.argmax(axis=1) )

array([[40,  0],
       [15, 21]])

In [36]:
#save model
import pickle
with open('model2_2.pkl', 'wb') as pickle_file:
    pickle.dump(model2_2, pickle_file)

INFO:tensorflow:Assets written to: ram://da8d22ca-763f-4e14-b1e4-7fe112022cd4/assets


In [37]:
#load model
import pickle
with open('model2_2.pkl', 'rb') as pickle_file:
    new_data = pickle.load(pickle_file)

In [38]:
predictions2_2test = new_data.predict(testX2_2, batch_size=BS)

In [39]:
print(classification_report(testY2_2.argmax(axis=1),	
                            predictions2_2test.argmax(axis=1),  
							target_names=le2_2.classes_ ))

              precision    recall  f1-score   support

      Normal       0.73      1.00      0.84        40
Tuberculosis       1.00      0.58      0.74        36

    accuracy                           0.80        76
   macro avg       0.86      0.79      0.79        76
weighted avg       0.86      0.80      0.79        76



# 3 class ==>  ***DenseNet201***

In [61]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [109]:
train_img_NorTBCa2 = []
for ImageName in tqdm(data_train3['ImageName']):                       # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = load_img( image_path )
    img = img_to_array(img)
    train_img_NorTBCa2.append(img)

100%|██████████| 451/451 [00:06<00:00, 69.40it/s] 


In [111]:
data3 = np.array(train_img_NorTBCa2 ) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


###แบ่งข้อมูล3

In [ ]:
train_img_NorTBCa = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))                                   #ขนาดของ input
    train_img_NorTBCa.append(img)

In [115]:
labels3 = np.array(data_train3[['NameType']] )        
labels3.shape = (len(labels3),) 

le3 = LabelEncoder()                                                                     
labels3 = le3.fit_transform(labels3)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3 = to_categorical(labels3, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]


In [116]:
(trainX3, testX3, trainY3, testY3) = train_test_split(data3, labels3,
	                                               test_size=0.25, random_state=40)

In [117]:
le3.classes_

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
trainX3_2=[]
for Ximg in trainX3:
    img = cv2.resize(Ximg, (224, 224) )
    trainX3.append(img)

## train3

In [85]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [97]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		rescale=1./255,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [87]:
base_model3 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [88]:
base_model3.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3 = tf.keras.models.Model(inputs=base_model3.input, outputs=prediction_layer)

In [89]:
model3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [119]:
H3 = model3.fit(
	x=aug.flow(trainX3, trainY3, batch_size=BS),
	validation_data=(testX3, testY3),
	steps_per_epoch=len(trainX3) // BS,
	epochs=10, callbacks=[callback])

ValueError: ignored

## test3

In [ ]:
predictions3 = model3.predict(testX3, batch_size=BS)

In [ ]:
predictions3[:10]

In [ ]:
testY3.argmax(axis=1)

In [ ]:
print(classification_report(testY3.argmax(axis=1),	
                           predictions3.argmax(axis=1),  
                            target_names=le3.classes_))

In [ ]:
confusion_matrix(testY3.argmax(axis=1),	
                        predictions3.argmax(axis=1) )

# 3 class ==> ***InceptionV3***

##แบ่งข้อมูล3
เลข 3 หมายถึงมี 3 คลาส

In [ ]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                   #ขนาดของ input
    train_img3_Incep.append(img)


100%|██████████| 180/180 [01:06<00:00,  2.71it/s]


In [ ]:
data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

In [ ]:
labels3_Incep = np.array(data_train3[['Type1']] )           #shape (180, 1) ค่าเป็น ชื่อคลาส Normal, Pneumonia , Lungcancer

In [ ]:
le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 --> ['Normal', 'Pneumonia', 'Lungcancer']
labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3)  มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
(trainX3_Incep, testX3_Incep, trainY3_Incep, testY3_Incep) = train_test_split(data3_Incep, labels3_Incep,
	                                        											test_size=0.25, random_state=52)

In [ ]:
le3_Incep.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

## train3

In [ ]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [ ]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

InceptionV3

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
base_model3_Incep.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_Incep = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

In [ ]:
model3_Incep.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    mode="max"
)

In [ ]:
H3_Incep = model3_Incep.fit(
	x=aug.flow(trainX3_Incep, trainY3_Incep, batch_size=BS),
	validation_data=(testX3_Incep, testY3_Incep),
	steps_per_epoch=len(trainX3_Incep) // BS,
	epochs=10, 
	callbacks = [callback])

Epoch 1/10
16/16 [==============================] - 46s 3s/step - loss: 0.7059 - accuracy: 0.4016 - val_loss: 0.6476 - val_accuracy: 0.4667
Epoch 2/10
16/16 [==============================] - 43s 3s/step - loss: 0.6019 - accuracy: 0.5354 - val_loss: 0.6036 - val_accuracy: 0.5333
Epoch 3/10
16/16 [==============================] - 52s 3s/step - loss: 0.5837 - accuracy: 0.5906 - val_loss: 0.5809 - val_accuracy: 0.6000
Epoch 4/10
16/16 [==============================] - 43s 3s/step - loss: 0.5727 - accuracy: 0.6220 - val_loss: 0.5571 - val_accuracy: 0.6889
Epoch 5/10
16/16 [==============================] - 40s 3s/step - loss: 0.5523 - accuracy: 0.6063 - val_loss: 0.5368 - val_accuracy: 0.7111
Epoch 6/10
16/16 [==============================] - 38s 2s/step - loss: 0.5350 - accuracy: 0.6378 - val_loss: 0.5182 - val_accuracy: 0.7333
Epoch 7/10
16/16 [==============================] - 38s 2s/step - loss: 0.5070 - accuracy: 0.6457 - val_loss: 0.4948 - val_accuracy: 0.7778
Epoch 8/10
16/16 [==

## test3

In [ ]:
predictions3_Incep = model3_Incep.predict(testX3_Incep, batch_size=BS)

In [ ]:
predictions3_Incep[:10]

array([[0.11190555, 0.44163662, 0.44645783],
       [0.35433874, 0.17010207, 0.47555923],
       [0.08360355, 0.6129521 , 0.30344427],
       [0.13412456, 0.11815269, 0.74772274],
       [0.07382458, 0.2461354 , 0.68004006],
       [0.06308495, 0.37355277, 0.5633623 ],
       [0.12921706, 0.30112725, 0.5696557 ],
       [0.0463132 , 0.7644238 , 0.18926297],
       [0.0321773 , 0.87751997, 0.09030275],
       [0.09041295, 0.25358486, 0.65600216]], dtype=float32)

In [ ]:
0.04993849+ 0.628473  + 0.32158852

1.00000001

In [ ]:
0.02641579+ 0.25785667+ 0.71572745

0.9999999100000001

In [ ]:
testY3_Incep.argmax(axis=1)

array([1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       2])

In [ ]:
print(classification_report(testY3_Incep.argmax(axis=1),	
                           predictions3_Incep.argmax(axis=1),  
                            target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.12      0.22         8
      Normal       0.74      0.94      0.83        18
   Pneumonia       0.81      0.89      0.85        19

    accuracy                           0.78        45
   macro avg       0.85      0.65      0.63        45
weighted avg       0.82      0.78      0.73        45



In [ ]:
le3_Incep.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

In [ ]:
confusion_matrix(testY3_Incep.argmax(axis=1),	
                        predictions3_Incep.argmax(axis=1) )

array([[ 1,  4,  3],
       [ 0, 17,  1],
       [ 0,  2, 17]])

## dump(save)/load sklearn model

In [ ]:
import joblib

In [ ]:
joblib.dump(model3_Incep, 'model3_Incep.joblib')  

INFO:tensorflow:Assets written to: ram://6c044177-9bf3-4f73-b85b-2c181b4d4d0a/assets


['model3_Incep.joblib']

In [ ]:
Incep= joblib.load('model3_Incep.joblib')

In [ ]:
Incep

In [ ]:
Incep.predict(testX3_Incep, batch_size=BS)[:10]

array([[0.11190555, 0.44163662, 0.44645783],
       [0.35433874, 0.17010207, 0.47555923],
       [0.08360355, 0.6129521 , 0.30344427],
       [0.13412456, 0.11815269, 0.74772274],
       [0.07382458, 0.2461354 , 0.68004006],
       [0.06308495, 0.37355277, 0.5633623 ],
       [0.12921706, 0.30112725, 0.5696557 ],
       [0.0463132 , 0.7644238 , 0.18926297],
       [0.0321773 , 0.87751997, 0.09030275],
       [0.09041295, 0.25358486, 0.65600216]], dtype=float32)

# Score CAM 
จาก https://arxiv.org/abs/1910.01279

Github https://github.com/haofanwang/Score-CAM

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from utils import *
from cam.scorecam import *

ModuleNotFoundError: ignored

In [ ]:
validation_normal_dir+ "/NORMAL2-IM-0117-0001.jpeg"

'/content/drive/My Drive/Senior Project/xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0117-0001.jpeg'

In [ ]:
os.listdir(validation_normal_dir)[:4]

['NORMAL2-IM-0117-0001.jpeg',
 'NORMAL2-IM-0059-0001.jpeg',
 'NORMAL2-IM-0123-0001.jpeg',
 'NORMAL2-IM-0120-0001.jpeg']

In [ ]:
im = cv2.imread(os.path.join(validation_normal_dir, 'NORMAL2-IM-0117-0001.jpeg' ))

In [ ]:
inceptionv3_scorecam = ScoreCAM(testX3_Incep)

input_image = load_image('images/'+'ILSVRC2012_val_00002193.JPEG')
input_ = apply_transforms(input_image)
if torch.cuda.is_available():
  input_ = input_.cuda()
predicted_class = resnet(input_).max(1)[-1]

scorecam_map = resnet_scorecam(input_)
basic_visualize(input_.cpu(), scorecam_map.type(torch.FloatTensor).cpu(),save_path='resnet.png')

ตัวอย่างจาก Github

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from utils import *
from cam.scorecam import *

In [ ]:
resnet = models.resnet18(pretrained=True).eval()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
resnet = models.resnet18(pretrained=True).eval()
resnet_model_dict = dict(type='resnet18', arch=resnet, layer_name='layer4',input_size=(224, 224))
resnet_scorecam = ScoreCAM(resnet_model_dict)

input_image = load_image('images/'+'ILSVRC2012_val_00002193.JPEG')
input_ = apply_transforms(input_image)
if torch.cuda.is_available():
  input_ = input_.cuda()
predicted_class = resnet(input_).max(1)[-1]

scorecam_map = resnet_scorecam(input_)
basic_visualize(input_.cpu(), scorecam_map.type(torch.FloatTensor).cpu(),save_path='resnet.png')